# Exploration of Yelp pretrained GPT model

In [4]:
import os
import sys

import torch
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

In [47]:
model_path = '../deep-latent-sequence-model/yelp_gpt/checkpoint-600/'
assert os.path.exists(model_path)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt = GPT2LMHeadModel.from_pretrained(model_path).eval()

In [48]:
tokenizer.tokenize('<|endoftext|>')

['<|endoftext|>']

In [49]:
sentence = "You can do whatever you want"
tokenizer.tokenize(sentence)

['You', 'Ġcan', 'Ġdo', 'Ġwhatever', 'Ġyou', 'Ġwant']

In [50]:
encoding = tokenizer.encode(sentence)
encoding

[1639, 460, 466, 4232, 345, 765]

In [51]:
tokenizer.decode(encoding)

'You can do whatever you want'

In [52]:
output = gpt(torch.tensor(encoding))
len(output)

2

In [53]:
logits = output[0]  # logits
logits.shape

torch.Size([6, 50257])

In [54]:
output[1].__len__()  # hidden states

12

In [61]:
preds = logits.argmax(1)
preds

tensor([423, 635, 326, 345, 765, 351], grad_fn=<NotImplemented>)

In [71]:
encoding

[1639, 460, 466, 4232, 345, 765]

In [77]:
words = tokenizer.tokenize(sentence)
for i in range(len(words)):
    beginning = tokenizer.decode(encoding[:i+1])
    predicted_continuation = tokenizer.decode(preds[i].item())
    print(beginning, '-->', predicted_continuation)

You -->  have
You can -->  also
You can do -->  that
You can do whatever -->  you
You can do whatever you -->  want
You can do whatever you want -->  with
